In [22]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import leaguedashplayerstats
pd.options.mode.chained_assignment = None

from natural import number

try:
    from nba_api.library.debug.debug import STATS_HEADERS
except ImportError:
    STATS_HEADERS = {
        'Host': 'stats.nba.com', 
        'Connection': 'keep-alive', 
        'Accept': 'application/json, text/plain, */*', 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
        'Referer': 'https://stats.nba.com/', 
        'Accept-Encoding': 'gzip, deflate, br', 
        'Accept-Language': 'en-US,en;q=0.9',
        'x-nba-stats-origin': 'stats',
        'x-nba-stats-token': 'true',
    }

In [23]:
def All_ShotChart_Table():

#   Pulling data from NBA API provided by https://github.com/swar
    
    shotchart = shotchartdetail.ShotChartDetail(player_id=0, season_type_all_star='Regular Season',
                                            team_id=0, season_nullable='2019-20', context_measure_simple='FGA', headers=STATS_HEADERS)

#   Creating Columns representing 14 specific spots on the court while 
#   also including total columns for 2pt, 3pt and Overall Total

    df = shotchart.get_data_frames()[0]

    AllChart = pd.DataFrame(df,columns=['PLAYER_NAME','TEAM_NAME', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
                                  'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE','SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG','LOC_X','LOC_Y','GAME_DATE','HTM','VTM'])

    AllChart1 = AllChart.pivot_table(index=['PLAYER_NAME'], columns='SHOT_TYPE', values=['SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG'], aggfunc=np.sum, margins=True)
    AllChart1.columns = AllChart1.columns.droplevel(level=0)

    columnsNameArr = AllChart1.columns.values
    columnsNameArr[0] = '2PT Field Goals Attempted'
    columnsNameArr[1] = '3PT Field Goals Attempted'
    columnsNameArr[2] = 'Total Attempts'
    columnsNameArr[3] = '2PT Field Goals Made'
    columnsNameArr[4] = '3PT Field Goals Made'
    columnsNameArr[5] = 'Total Made'

    AllChart2 = AllChart.pivot_table(index=['PLAYER_NAME'], columns=['SHOT_TYPE','SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA'], values=['SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG'], aggfunc=np.sum, margins=True)

    AllChart2.columns = AllChart2.columns.map(' '.join).str.strip(' ')

    AllChart3 = AllChart2[['SHOT_MADE_FLAG 2PT Field Goal In The Paint (Non-RA) Center(C)', 'SHOT_MADE_FLAG 2PT Field Goal In The Paint (Non-RA) Left Side(L)', 'SHOT_MADE_FLAG 2PT Field Goal In The Paint (Non-RA) Right Side(R)',
     'SHOT_ATTEMPTED_FLAG 2PT Field Goal In The Paint (Non-RA) Center(C)','SHOT_ATTEMPTED_FLAG 2PT Field Goal In The Paint (Non-RA) Left Side(L)','SHOT_ATTEMPTED_FLAG 2PT Field Goal In The Paint (Non-RA) Right Side(R)',
     'SHOT_MADE_FLAG 2PT Field Goal Mid-Range Center(C)','SHOT_MADE_FLAG 2PT Field Goal Mid-Range Left Side Center(LC)','SHOT_MADE_FLAG 2PT Field Goal Mid-Range Left Side(L)','SHOT_MADE_FLAG 2PT Field Goal Mid-Range Right Side Center(RC)',
     'SHOT_MADE_FLAG 2PT Field Goal Mid-Range Right Side(R)','SHOT_ATTEMPTED_FLAG 2PT Field Goal Mid-Range Center(C)','SHOT_ATTEMPTED_FLAG 2PT Field Goal Mid-Range Left Side Center(LC)','SHOT_ATTEMPTED_FLAG 2PT Field Goal Mid-Range Left Side(L)',
     'SHOT_ATTEMPTED_FLAG 2PT Field Goal Mid-Range Right Side Center(RC)','SHOT_ATTEMPTED_FLAG 2PT Field Goal Mid-Range Right Side(R)','SHOT_MADE_FLAG 2PT Field Goal Restricted Area Center(C)','SHOT_ATTEMPTED_FLAG 2PT Field Goal Restricted Area Center(C)',
     'SHOT_MADE_FLAG 3PT Field Goal Above the Break 3 Center(C)','SHOT_MADE_FLAG 3PT Field Goal Above the Break 3 Left Side Center(LC)','SHOT_MADE_FLAG 3PT Field Goal Above the Break 3 Right Side Center(RC)','SHOT_ATTEMPTED_FLAG 3PT Field Goal Above the Break 3 Center(C)',
     'SHOT_ATTEMPTED_FLAG 3PT Field Goal Above the Break 3 Left Side Center(LC)','SHOT_ATTEMPTED_FLAG 3PT Field Goal Above the Break 3 Right Side Center(RC)','SHOT_MADE_FLAG 3PT Field Goal Left Corner 3 Left Side(L)','SHOT_ATTEMPTED_FLAG 3PT Field Goal Left Corner 3 Left Side(L)',
     'SHOT_MADE_FLAG 3PT Field Goal Right Corner 3 Right Side(R)','SHOT_ATTEMPTED_FLAG 3PT Field Goal Right Corner 3 Right Side(R)']]

    AllChartDetail_Filtered = pd.merge(AllChart1, AllChart3, on='PLAYER_NAME')

    columnsNameChange = AllChartDetail_Filtered.columns.values

    columnsNameChange[6] = 'In The Paint (Non-RA) 2PT Center(C) Made'
    columnsNameChange[7] = 'In The Paint (Non-RA) 2PT Left Side(L) Made'
    columnsNameChange[8] = 'In The Paint (Non-RA) 2PT Right Side(R) Made'
    columnsNameChange[9] = 'In The Paint (Non-RA) 2PT Center(C) Attempted'
    columnsNameChange[10] = 'In The Paint (Non-RA) 2PT Left Side(L) Attempted'
    columnsNameChange[11] = 'In The Paint (Non-RA) 2PT Right Side(R) Attempted'
    columnsNameChange[12] = 'Mid-Range 2PT Center(C) Made'
    columnsNameChange[13] = 'Mid-Range 2PT Left Side Center(LC) Made'
    columnsNameChange[14] = 'Mid-Range 2PT Left Side(L) Made'
    columnsNameChange[15] = 'Mid-Range 2PT Right Side Center(RC) Made'
    columnsNameChange[16] = 'Mid-Range 2PT Right Side(R) Made'
    columnsNameChange[17] = 'Mid-Range 2PT Center(C) Attempted'
    columnsNameChange[18] = 'Mid-Range 2PT Left Side Center(LC) Attempted'
    columnsNameChange[19] = 'Mid-Range 2PT Left Side(L) Attempted'
    columnsNameChange[20] = 'Mid-Range 2PT Right Side Center(RC) Attempted'
    columnsNameChange[21] = 'Mid-Range 2PT Right Side(R) Attempted'
    columnsNameChange[22] = 'Restricted Area 2PT  Center(C) Made'
    columnsNameChange[23] = 'Restricted Area 2PT Center(C) Attempted'
    columnsNameChange[24] = 'Above the Break 3PT Center(C) Made'
    columnsNameChange[25] = 'Above the Break 3PT Left Side Center(LC) Made'
    columnsNameChange[26] = 'Above the Break 3PT Right Side Center(RC) Made'
    columnsNameChange[27] = 'Above the Break 3PT Center(C) Attempted'
    columnsNameChange[28] = 'Above the Break 3PT Left Side Center(LC) Attempted'
    columnsNameChange[29] = 'Above the Break 3PT Right Side Center(RC) Attempted'
    columnsNameChange[30] = 'Left Corner 3PT Left Side(L) Made'      
    columnsNameChange[31] = 'Left Corner 3PT Left Side(L) Attempted'
    columnsNameChange[32] = 'Right Corner 3PT Right Side(R) Made'        
    columnsNameChange[33] = 'Right Corner 3PT Right Side(R) Attempted'

    AllChartDetail_Filtered.columns=['2PT Field Goals Attempted',
                     '3PT Field Goals Attempted',
                     'Total Attempts',
                     '2PT Field Goals Made',
                     '3PT Field Goals Made',
                     'Total Made',
                     'In The Paint (Non-RA) 2PT Center(C) Made',
                     'In The Paint (Non-RA) 2PT Left Side(L) Made',
                     'In The Paint (Non-RA) 2PT Right Side(R) Made',
                     'In The Paint (Non-RA) 2PT Center(C) Attempted',
                     'In The Paint (Non-RA) 2PT Left Side(L) Attempted',
                     'In The Paint (Non-RA) 2PT Right Side(R) Attempted',
                     'Mid-Range 2PT Center(C) Made',
                     'Mid-Range 2PT Left Side Center(LC) Made',
                     'Mid-Range 2PT Left Side(L) Made',
                     'Mid-Range 2PT Right Side Center(RC) Made',
                     'Mid-Range 2PT Right Side(R) Made',
                     'Mid-Range 2PT Center(C) Attempted',
                     'Mid-Range 2PT Left Side Center(LC) Attempted',
                     'Mid-Range 2PT Left Side(L) Attempted',
                     'Mid-Range 2PT Right Side Center(RC) Attempted',
                     'Mid-Range 2PT Right Side(R) Attempted',
                     'Restricted Area 2PT Center(C) Made',
                     'Restricted Area 2PT Center(C) Attempted',
                     'Above the Break 3PT Center(C) Made',
                     'Above the Break 3PT Left Side Center(LC) Made',
                     'Above the Break 3PT Right Side Center(RC) Made',
                     'Above the Break 3PT Center(C) Attempted',
                     'Above the Break 3PT Left Side Center(LC) Attempted',
                     'Above the Break 3PT Right Side Center(RC) Attempted',
                     'Left Corner 3PT Left Side(L) Made',
                     'Left Corner 3PT Left Side(L) Attempted',
                     'Right Corner 3PT Right Side(R) Made',
                     'Right Corner 3PT Right Side(R) Attempted']

    AllChartDetail_Filtered['In The Paint (Non-RA) Center_%'] = AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Center(C) Made']/AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Center(C) Attempted']
    AllChartDetail_Filtered['In The Paint (Non-RA) Left_%'] = AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Left Side(L) Made']/AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Left Side(L) Attempted']
    AllChartDetail_Filtered['In The Paint (Non-RA) Right_%'] = AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Right Side(R) Made']/AllChartDetail_Filtered['In The Paint (Non-RA) 2PT Right Side(R) Attempted']
    AllChartDetail_Filtered['Mid-Range Center_%'] = AllChartDetail_Filtered['Mid-Range 2PT Center(C) Made']/AllChartDetail_Filtered['Mid-Range 2PT Center(C) Attempted']
    AllChartDetail_Filtered['Mid-Range Left Center_%'] = AllChartDetail_Filtered['Mid-Range 2PT Left Side Center(LC) Made']/AllChartDetail_Filtered['Mid-Range 2PT Left Side Center(LC) Attempted']
    AllChartDetail_Filtered['Mid-Range Left_%'] = AllChartDetail_Filtered['Mid-Range 2PT Left Side(L) Made']/AllChartDetail_Filtered['Mid-Range 2PT Left Side(L) Attempted']
    AllChartDetail_Filtered['Mid-Range Right Center_%'] = AllChartDetail_Filtered['Mid-Range 2PT Right Side Center(RC) Made']/AllChartDetail_Filtered['Mid-Range 2PT Right Side Center(RC) Attempted']
    AllChartDetail_Filtered['Mid-Range Right_%'] = AllChartDetail_Filtered['Mid-Range 2PT Right Side(R) Made']/AllChartDetail_Filtered['Mid-Range 2PT Right Side(R) Attempted']
    AllChartDetail_Filtered['Restricted Area_%'] = AllChartDetail_Filtered['Restricted Area 2PT Center(C) Made']/AllChartDetail_Filtered['Restricted Area 2PT Center(C) Attempted']
    AllChartDetail_Filtered['3PT Above the Break Center_%'] = AllChartDetail_Filtered['Above the Break 3PT Center(C) Made']/AllChartDetail_Filtered['Above the Break 3PT Center(C) Attempted']
    AllChartDetail_Filtered['3PT Above the Break Left Center_%'] = AllChartDetail_Filtered['Above the Break 3PT Left Side Center(LC) Made']/AllChartDetail_Filtered['Above the Break 3PT Left Side Center(LC) Attempted']
    AllChartDetail_Filtered['3PT Above the Break Right Center_%'] = AllChartDetail_Filtered['Above the Break 3PT Right Side Center(RC) Made']/AllChartDetail_Filtered['Above the Break 3PT Right Side Center(RC) Attempted']
    AllChartDetail_Filtered['3PT Left Corner_%'] = AllChartDetail_Filtered['Left Corner 3PT Left Side(L) Made']/AllChartDetail_Filtered['Left Corner 3PT Left Side(L) Attempted']
    AllChartDetail_Filtered['3PT Right Corner_%'] = AllChartDetail_Filtered['Right Corner 3PT Right Side(R) Made']/AllChartDetail_Filtered['Right Corner 3PT Right Side(R) Attempted']
    AllChartDetail_Filtered['2PT_%'] = AllChartDetail_Filtered['2PT Field Goals Made']/AllChartDetail_Filtered['2PT Field Goals Attempted']
    AllChartDetail_Filtered['3PT_%'] = AllChartDetail_Filtered['3PT Field Goals Made']/AllChartDetail_Filtered['3PT Field Goals Attempted']
    AllChartDetail_Filtered['Total_%'] = AllChartDetail_Filtered['Total Made']/AllChartDetail_Filtered['Total Attempts']

    AllChartDetail_Final_1 = AllChartDetail_Filtered.sort_values(by='Total Attempts', ascending=False)

    AllChartDetail_Final_2 = AllChartDetail_Final_1.replace(np.nan, 0)
    AllChartDetail_Final_3 = AllChartDetail_Final_2.round(2)       


#     Created the Columns for Percentile Rank & League Rank for each location.
#     For each location, I pulled the average attempts and only included those players who
#     have attempted at least that average. The final chart will only include those players who attempted
#     at least the average for total shot attempts
    

#     Total Percentile Rank/League Rank Column Creation
    
    total_nba_shots = AllChartDetail_Final_3['Total Attempts']['All']
    reindex_AllChartDetail_Final_3 = AllChartDetail_Final_3.reset_index()
    total_nba_players = len(reindex_AllChartDetail_Final_3['PLAYER_NAME'])-1
    average_total_shots = total_nba_shots/total_nba_players
    
    total_pt = pd.DataFrame(AllChartDetail_Final_3,columns=['Total Made', 'Total Attempts', 'Total_%'])
    total_pt1 = total_pt.loc[total_pt['Total Attempts'] >= average_total_shots]
    total_pt1['Total_Percentile']=(total_pt1['Total_%'].rank(pct=True) * 100).round()
    total_pt1['Total_League Rank']=total_pt1['Total_%'].rank(ascending=False,pct=False)
    
    
#     Paint Center Percentile Rank/League Rank Column Creation
    
    Paint_Non_RA_nba_shots = AllChartDetail_Final_3['In The Paint (Non-RA) 2PT Center(C) Attempted']['All']
    average_Paint_Non_RA_nba_shots = Paint_Non_RA_nba_shots/total_nba_players    
    
    Paint_Non_RA = pd.DataFrame(AllChartDetail_Final_3,columns=['In The Paint (Non-RA) 2PT Center(C) Made', 'In The Paint (Non-RA) 2PT Center(C) Attempted', 'In The Paint (Non-RA) Center_%'])
    Paint_Non_RA_2 = Paint_Non_RA.loc[Paint_Non_RA[ 'In The Paint (Non-RA) 2PT Center(C) Attempted'] >= average_Paint_Non_RA_nba_shots]
    Paint_Non_RA_2['In The Paint (Non-RA) Center_Percentile']=(Paint_Non_RA_2['In The Paint (Non-RA) Center_%'].rank(pct=True) * 100).round()
    Paint_Non_RA_2['In The Paint (Non-RA) Center_League Rank']=Paint_Non_RA_2['In The Paint (Non-RA) Center_%'].rank(ascending=False,pct=False)


#     Paint Left Percentile Rank/League Rank Column Creation
    
    Paint_Non_RA_Left_nba_shots = AllChartDetail_Final_3['In The Paint (Non-RA) 2PT Left Side(L) Attempted']['All']
    average_Paint_Non_RA_Left_shots = Paint_Non_RA_Left_nba_shots/total_nba_players    
    
    Paint_Non_RA_Left = pd.DataFrame(AllChartDetail_Final_3,columns=['In The Paint (Non-RA) 2PT Left Side(L) Made', 'In The Paint (Non-RA) 2PT Left Side(L) Attempted', 'In The Paint (Non-RA) Left_%'])
    Paint_Non_RA_Left1 = Paint_Non_RA_Left.loc[Paint_Non_RA_Left[ 'In The Paint (Non-RA) 2PT Left Side(L) Attempted'] >= average_Paint_Non_RA_Left_shots]
    Paint_Non_RA_Left1['In The Paint (Non-RA) Left_Percentile']=(Paint_Non_RA_Left1['In The Paint (Non-RA) Left_%'].rank(pct=True) * 100).round()
    Paint_Non_RA_Left1['In The Paint (Non-RA) Left_League Rank']=Paint_Non_RA_Left1['In The Paint (Non-RA) Left_%'].rank(ascending=False,pct=False)

#     Paint Right Percentile Rank/League Rank Column Creation
    
    Paint_Non_RA_Right_nba_shots = AllChartDetail_Final_3['In The Paint (Non-RA) 2PT Right Side(R) Attempted']['All']
    average_Paint_Non_RA_Right_shots = Paint_Non_RA_Right_nba_shots/total_nba_players    
    
    Paint_Non_RA_Right = pd.DataFrame(AllChartDetail_Final_3,columns=['In The Paint (Non-RA) 2PT Right Side(R) Made', 'In The Paint (Non-RA) 2PT Right Side(R) Attempted', 'In The Paint (Non-RA) Right_%'])
    Paint_Non_RA_Right1 = Paint_Non_RA_Right.loc[Paint_Non_RA_Right[ 'In The Paint (Non-RA) 2PT Right Side(R) Attempted'] >= average_Paint_Non_RA_Right_shots]
    Paint_Non_RA_Right1['In The Paint (Non-RA) Right_Percentile']=(Paint_Non_RA_Right1['In The Paint (Non-RA) Right_%'].rank(pct=True) * 100).round()
    Paint_Non_RA_Right1['In The Paint (Non-RA) Right_League Rank']=Paint_Non_RA_Right1['In The Paint (Non-RA) Right_%'].rank(ascending=False,pct=False)

#     Restricted Area Percentile Rank/League Rank Column Creation
    
    Restricted_Area_nba_shots = AllChartDetail_Final_3['Restricted Area 2PT Center(C) Attempted']['All']
    average_Restricted_Area_shots = Restricted_Area_nba_shots/total_nba_players    
    
    Restricted_Area = pd.DataFrame(AllChartDetail_Final_3,columns=['Restricted Area 2PT Center(C) Made', 'Restricted Area 2PT Center(C) Attempted', 'Restricted Area_%'])
    Restricted_Area1 = Restricted_Area.loc[Restricted_Area['Restricted Area 2PT Center(C) Attempted'] >= average_Restricted_Area_shots]
    Restricted_Area1['Restricted Area_Percentile']=(Restricted_Area1['Restricted Area_%'].rank(pct=True) * 100).round()
    Restricted_Area1['Restricted Area_League Rank']=Restricted_Area1['Restricted Area_%'].rank(ascending=False,pct=False)

#     Midrange Center Percentile Rank/League Rank Column Creation

    Midrange_Center_nba_shots = AllChartDetail_Final_3['Mid-Range 2PT Center(C) Attempted']['All']
    average_Midrange_Center_shots = Midrange_Center_nba_shots/total_nba_players    
    
    Midrange_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Mid-Range 2PT Center(C) Made', 'Mid-Range 2PT Center(C) Attempted', 'Mid-Range Center_%'])
    Midrange_Center1 = Midrange_Center.loc[Midrange_Center['Mid-Range 2PT Center(C) Attempted'] >= average_Midrange_Center_shots]
    Midrange_Center1['Mid-Range Center_Percentile']=(Midrange_Center1['Mid-Range Center_%'].rank(pct=True) * 100).round()
    Midrange_Center1['Mid-Range Center_League Rank']=Midrange_Center1['Mid-Range Center_%'].rank(ascending=False,pct=False)
    
#     Midrange Left Center Percentile Rank/League Rank Column Creation
    
    Midrange_Left_Center_nba_shots = AllChartDetail_Final_3['Mid-Range 2PT Left Side Center(LC) Attempted']['All']
    average_Midrange_Left_Center_shots = Midrange_Left_Center_nba_shots/total_nba_players    
    
    Midrange_Left_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Mid-Range 2PT Left Side Center(LC) Made', 'Mid-Range 2PT Left Side Center(LC) Attempted', 'Mid-Range Left Center_%'])
    Midrange_Left_Center1 = Midrange_Left_Center.loc[Midrange_Left_Center['Mid-Range 2PT Left Side Center(LC) Attempted'] >= average_Midrange_Left_Center_shots]
    Midrange_Left_Center1['Mid-Range Left Center_Percentile']=(Midrange_Left_Center1['Mid-Range Left Center_%'].rank(pct=True) * 100).round()                
    Midrange_Left_Center1['Mid-Range Left Center_League Rank']=Midrange_Left_Center1['Mid-Range Left Center_%'].rank(ascending=False,pct=False)

    
#     Midrange Left Percentile Rank/League Rank Column Creation   
    
    Midrange_Left_nba_shots = AllChartDetail_Final_3['Mid-Range 2PT Left Side(L) Attempted']['All']
    average_Midrange_Left_shots = Midrange_Left_nba_shots/total_nba_players    
    
    Midrange_Left = pd.DataFrame(AllChartDetail_Final_3,columns=['Mid-Range 2PT Left Side(L) Made', 'Mid-Range 2PT Left Side(L) Attempted', 'Mid-Range Left_%'])
    Midrange_Left1 = Midrange_Left.loc[Midrange_Left['Mid-Range 2PT Left Side(L) Attempted'] >= average_Midrange_Left_shots]
    Midrange_Left1['Mid-Range Left_Percentile']=(Midrange_Left1['Mid-Range Left_%'].rank(pct=True) * 100).round()
    Midrange_Left1['Mid-Range Left_League Rank']=Midrange_Left1['Mid-Range Left_%'].rank(ascending=False,pct=False)
    
#     Midrange Right Center Percentile Rank/League Rank Column Creation    
    
    Midrange_Right_Center_nba_shots = AllChartDetail_Final_3['Mid-Range 2PT Right Side Center(RC) Attempted']['All']
    average_Midrange_Right_Center_shots = Midrange_Right_Center_nba_shots/total_nba_players    
    
    Midrange_Right_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Mid-Range 2PT Right Side Center(RC) Made', 'Mid-Range 2PT Right Side Center(RC) Attempted', 'Mid-Range Right Center_%'])
    Midrange_Right_Center1 = Midrange_Right_Center.loc[Midrange_Right_Center['Mid-Range 2PT Right Side Center(RC) Attempted'] >= average_Midrange_Right_Center_shots]
    Midrange_Right_Center1['Mid-Range Right Center_Percentile']= (Midrange_Right_Center1['Mid-Range Right Center_%'].rank(pct=True) * 100).round()                    
    Midrange_Right_Center1['Mid-Range Right Center_League Rank']= Midrange_Right_Center1['Mid-Range Right Center_%'].rank(ascending=False,pct=False)

#     Midrange Right Percentile Rank/League Rank Column Creation

    Midrange_Right_nba_shots = AllChartDetail_Final_3['Mid-Range 2PT Right Side(R) Attempted']['All']
    average_Midrange_Right_shots = Midrange_Right_nba_shots/total_nba_players    
    
    Midrange_Right = pd.DataFrame(AllChartDetail_Final_3,columns=['Mid-Range 2PT Right Side(R) Made', 'Mid-Range 2PT Right Side(R) Attempted', 'Mid-Range Right_%'])
    Midrange_Right1 = Midrange_Right.loc[Midrange_Right['Mid-Range 2PT Right Side(R) Attempted'] >= average_Midrange_Right_shots]
    Midrange_Right1['Mid-Range Right_Percentile']= (Midrange_Right1['Mid-Range Right_%'].rank(pct=True) * 100).round()
    Midrange_Right1['Mid-Range Right_League Rank']= Midrange_Right1['Mid-Range Right_%'].rank(ascending=False,pct=False)

#     Above The Break Center Percentile Rank/League Rank Column Creation
    
    Above_Break_Center_nba_shots = AllChartDetail_Final_3['Above the Break 3PT Center(C) Attempted']['All']
    average_Above_Break_Center_shots = Above_Break_Center_nba_shots/total_nba_players    
    
    Above_Break_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Above the Break 3PT Center(C) Made', 'Above the Break 3PT Center(C) Attempted', '3PT Above the Break Center_%'])
    Above_Break_Center1 = Above_Break_Center.loc[Above_Break_Center['Above the Break 3PT Center(C) Attempted'] >= average_Above_Break_Center_shots]
    Above_Break_Center1['3PT Above the Break Center_Percentile']= (Above_Break_Center1['3PT Above the Break Center_%'].rank(pct=True) * 100).round()
    Above_Break_Center1['3PT Above the Break Center_League Rank']=Above_Break_Center1['3PT Above the Break Center_%'].rank(ascending=False,pct=False)

#     Above The Break Left Center Percentile Rank/League Rank Column Creation 
    
    Above_Break_Left_Center_nba_shots = AllChartDetail_Final_3['Above the Break 3PT Left Side Center(LC) Attempted']['All']
    average_Above_Break_Left_Center_shots = Above_Break_Left_Center_nba_shots/total_nba_players    
    
    Above_Break_Left_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Above the Break 3PT Left Side Center(LC) Made', 'Above the Break 3PT Left Side Center(LC) Attempted', '3PT Above the Break Left Center_%'])
    Above_Break_Left_Center1 = Above_Break_Left_Center.loc[Above_Break_Left_Center['Above the Break 3PT Left Side Center(LC) Attempted'] >= average_Above_Break_Left_Center_shots]
    Above_Break_Left_Center1['3PT Above the Break Left Center_Percentile']= (Above_Break_Left_Center1['3PT Above the Break Left Center_%'].rank(pct=True) * 100).round()      
    Above_Break_Left_Center1['3PT Above the Break Left Center_League Rank']=Above_Break_Left_Center1['3PT Above the Break Left Center_%'].rank(ascending=False,pct=False)

#     Above The Break Right Center Percentile Rank/League Rank Column Creation
    
    Above_Break_Right_Center_nba_shots = AllChartDetail_Final_3['Above the Break 3PT Right Side Center(RC) Attempted']['All']
    average_Above_Break_Right_Center_shots = Above_Break_Right_Center_nba_shots/total_nba_players    
    
    Above_Break_Right_Center = pd.DataFrame(AllChartDetail_Final_3,columns=['Above the Break 3PT Right Side Center(RC) Made', 'Above the Break 3PT Right Side Center(RC) Attempted', '3PT Above the Break Right Center_%'])
    Above_Break_Right_Center1 = Above_Break_Right_Center.loc[Above_Break_Right_Center['Above the Break 3PT Right Side Center(RC) Attempted'] >= average_Above_Break_Right_Center_shots]
    Above_Break_Right_Center1['3PT Above the Break Right Center_Percentile']= (Above_Break_Right_Center1['3PT Above the Break Right Center_%'].rank(pct=True) * 100).round()            
    Above_Break_Right_Center1['3PT Above the Break Right Center_League Rank']= Above_Break_Right_Center1['3PT Above the Break Right Center_%'].rank(ascending=False,pct=False)

#     Left Corner Three Percentile Rank/League Rank Column Creation  
    
    Left_Corner_nba_shots = AllChartDetail_Final_3['Left Corner 3PT Left Side(L) Attempted']['All']
    average_Left_Corner_shots = Left_Corner_nba_shots/total_nba_players    
    
    Left_Corner = pd.DataFrame(AllChartDetail_Final_3,columns=['Left Corner 3PT Left Side(L) Made', 'Left Corner 3PT Left Side(L) Attempted', '3PT Left Corner_%'])
    Left_Corner1 = Left_Corner.loc[Left_Corner['Left Corner 3PT Left Side(L) Attempted'] >= average_Left_Corner_shots]
    Left_Corner1['3PT Left Corner_Percentile']= (Left_Corner1['3PT Left Corner_%'].rank(pct=True) * 100).round()                        
    Left_Corner1['3PT Left Corner_League Rank']=Left_Corner1['3PT Left Corner_%'].rank(ascending=False,pct=False)

#     Right Corner Three Percentile Rank/League Rank Column Creation    
    
    Right_Corner_nba_shots = AllChartDetail_Final_3['Right Corner 3PT Right Side(R) Attempted']['All']
    average_Right_Corner_shots = Right_Corner_nba_shots/total_nba_players    
    
    Right_Corner = pd.DataFrame(AllChartDetail_Final_3,columns=['Right Corner 3PT Right Side(R) Made', 'Right Corner 3PT Right Side(R) Attempted', '3PT Right Corner_%'])
    Right_Corner1 = Right_Corner.loc[Right_Corner['Right Corner 3PT Right Side(R) Attempted'] >= average_Right_Corner_shots]
    Right_Corner1['3PT Right Corner_Percentile']= (Right_Corner1['3PT Right Corner_%'].rank(pct=True) * 100).round()                         
    Right_Corner1['3PT Right Corner_League Rank']=Right_Corner1['3PT Right Corner_%'].rank(ascending=False,pct=False)

#     Two Point Field Goal Percentile Rank/League Rank Column Creation
    
    two_pt_nba_shots = AllChartDetail_Final_3['2PT Field Goals Attempted']['All']
    average_two_pt_shots = two_pt_nba_shots/total_nba_players    
    
    two_pt = pd.DataFrame(AllChartDetail_Final_3,columns=['2PT Field Goals Made', '2PT Field Goals Attempted', '2PT_%'])
    two_pt1 = two_pt.loc[two_pt['2PT Field Goals Attempted'] >= average_two_pt_shots]
    two_pt1['2PT_Percentile']= (two_pt1['2PT_%'].rank(pct=True) * 100).round()                          
    two_pt1['2PT_League Rank']=two_pt1['2PT_%'].rank(ascending=False,pct=False)

#     Three Point Field Goal Percentile Rank/League Rank Column Creation 
    
    three_pt_nba_shots = AllChartDetail_Final_3['3PT Field Goals Attempted']['All']
    average_three_pt_shots = three_pt_nba_shots/total_nba_players    
    
    three_pt = pd.DataFrame(AllChartDetail_Final_3,columns=['3PT Field Goals Made', '3PT Field Goals Attempted', '3PT_%'])
    three_pt1 = three_pt.loc[three_pt['3PT Field Goals Attempted'] >= average_three_pt_shots]
    three_pt1['3PT_Percentile']= (three_pt1['3PT_%'].rank(pct=True) * 100).round()                                       
    three_pt1['3PT_League Rank']=three_pt1['3PT_%'].rank(ascending=False,pct=False)


    Result = pd.concat([Paint_Non_RA_2, Paint_Non_RA_Left1, Paint_Non_RA_Right1, Restricted_Area1, Midrange_Center1, Midrange_Left_Center1, Midrange_Left1, Midrange_Right_Center1, Midrange_Right1, Above_Break_Center1, Above_Break_Left_Center1,
                   Above_Break_Right_Center1, Left_Corner1, Right_Corner1, two_pt1, three_pt1, total_pt1], axis=1, sort=False).reindex(Paint_Non_RA_2.index)


    AllPaintMerge = AllChartDetail_Final_3.merge(Result[['In The Paint (Non-RA) Center_Percentile', 'In The Paint (Non-RA) Left_Percentile', 'In The Paint (Non-RA) Right_Percentile', 
                                                     'Restricted Area_Percentile', 'Mid-Range Center_Percentile', 'Mid-Range Left Center_Percentile', 
                                                     'Mid-Range Left_Percentile', 'Mid-Range Right Center_Percentile', 
                                                    'Mid-Range Right_Percentile', '3PT Above the Break Center_Percentile', '3PT Above the Break Left Center_Percentile',
                                                    '3PT Above the Break Right Center_Percentile', '3PT Left Corner_Percentile', '3PT Right Corner_Percentile',
                                                    '2PT_Percentile', '3PT_Percentile', 'Total_Percentile','Total_League Rank','3PT_League Rank','2PT_League Rank','3PT Right Corner_League Rank','3PT Left Corner_League Rank',
                                                    '3PT Above the Break Right Center_League Rank','3PT Above the Break Left Center_League Rank','3PT Above the Break Center_League Rank','Mid-Range Right_League Rank','Mid-Range Right Center_League Rank',
                                                    'Mid-Range Left_League Rank','Mid-Range Left Center_League Rank','Mid-Range Center_League Rank','Restricted Area_League Rank','In The Paint (Non-RA) Right_League Rank','In The Paint (Non-RA) Left_League Rank',
                                                    'In The Paint (Non-RA) Center_League Rank']], on='PLAYER_NAME', how='left')
    
    Final = AllPaintMerge[['In The Paint (Non-RA) 2PT Center(C) Made',
                        'In The Paint (Non-RA) 2PT Center(C) Attempted',
                        'In The Paint (Non-RA) Center_%',
                        'In The Paint (Non-RA) Center_Percentile',
                        'In The Paint (Non-RA) Center_League Rank',
                        'In The Paint (Non-RA) 2PT Left Side(L) Made',
                        'In The Paint (Non-RA) 2PT Left Side(L) Attempted',
                        'In The Paint (Non-RA) Left_%',
                        'In The Paint (Non-RA) Left_Percentile',
                        'In The Paint (Non-RA) Left_League Rank',
                        'In The Paint (Non-RA) 2PT Right Side(R) Made',
                        'In The Paint (Non-RA) 2PT Right Side(R) Attempted',
                        'In The Paint (Non-RA) Right_%',
                        'In The Paint (Non-RA) Right_Percentile',
                        'In The Paint (Non-RA) Right_League Rank',
                        'Mid-Range 2PT Center(C) Made',
                        'Mid-Range 2PT Center(C) Attempted',
                        'Mid-Range Center_%',
                        'Mid-Range Center_Percentile',
                        'Mid-Range Center_League Rank',
                        'Mid-Range 2PT Left Side Center(LC) Made',
                        'Mid-Range 2PT Left Side Center(LC) Attempted',
                        'Mid-Range Left Center_%',
                        'Mid-Range Left Center_Percentile',
                        'Mid-Range Left Center_League Rank',
                        'Mid-Range 2PT Left Side(L) Made',
                        'Mid-Range 2PT Left Side(L) Attempted',
                        'Mid-Range Left_%',
                        'Mid-Range Left_Percentile',
                        'Mid-Range Left_League Rank',
                        'Mid-Range 2PT Right Side Center(RC) Made',
                        'Mid-Range 2PT Right Side Center(RC) Attempted',
                        'Mid-Range Right Center_%',
                        'Mid-Range Right Center_Percentile',
                        'Mid-Range Right Center_League Rank',
                        'Mid-Range 2PT Right Side(R) Made',
                        'Mid-Range 2PT Right Side(R) Attempted',
                        'Mid-Range Right_%',
                        'Mid-Range Right_Percentile',
                        'Mid-Range Right_League Rank',
                        'Restricted Area 2PT Center(C) Made',
                        'Restricted Area 2PT Center(C) Attempted',
                        'Restricted Area_%',
                        'Restricted Area_Percentile',
                        'Restricted Area_League Rank',
                        'Above the Break 3PT Center(C) Made',
                        'Above the Break 3PT Center(C) Attempted',
                        '3PT Above the Break Center_%',
                        '3PT Above the Break Center_Percentile',
                        '3PT Above the Break Center_League Rank',
                        'Above the Break 3PT Left Side Center(LC) Made',
                        'Above the Break 3PT Left Side Center(LC) Attempted',
                        '3PT Above the Break Left Center_%',
                        '3PT Above the Break Left Center_Percentile',
                        '3PT Above the Break Left Center_League Rank',
                        'Above the Break 3PT Right Side Center(RC) Made',
                        'Above the Break 3PT Right Side Center(RC) Attempted',
                        '3PT Above the Break Right Center_%',
                        '3PT Above the Break Right Center_Percentile',
                        '3PT Above the Break Right Center_League Rank',
                        'Left Corner 3PT Left Side(L) Made',
                        'Left Corner 3PT Left Side(L) Attempted',
                        '3PT Left Corner_%',
                        '3PT Left Corner_Percentile',
                        '3PT Left Corner_League Rank',
                        'Right Corner 3PT Right Side(R) Made',
                        'Right Corner 3PT Right Side(R) Attempted',
                        '3PT Right Corner_%',
                        '3PT Right Corner_Percentile',
                        '3PT Right Corner_League Rank',
                        '2PT Field Goals Made',
                        '2PT Field Goals Attempted',
                        '2PT_%',
                        '2PT_Percentile',
                        '2PT_League Rank',
                        '3PT Field Goals Made',
                        '3PT Field Goals Attempted',
                        '3PT_%',
                        '3PT_Percentile',
                        '3PT_League Rank',
                        'Total Made',
                        'Total Attempts',
                        'Total_%',
                        'Total_Percentile',
                        'Total_League Rank']]

    Final_1 = Final.loc[Final[ 'Total Attempts'] >= average_total_shots]
    
    return Final_1

#   Created the below function to highlight those players who are in the Top Ten for the league in each location

def custom_style(row):
   
    style = ['background-color: white']*len(row)
    
    if (row[('In The Paint (Non-RA) Center','League Rank')] <= 10) & (row[('In The Paint (Non-RA) Center','League Rank')] > 0):
        style[2] = 'background-color: gold'    
    else:
        style[2] = 'background-color: transparent'        
    
    if (row[('In The Paint (Non-RA) Left','League Rank')] <= 10) & (row[('In The Paint (Non-RA) Left','League Rank')] > 0):
        style[7] = 'background-color: gold'
    else:
        style[7] = 'background-color: transparent'
    
    if (row[('In The Paint (Non-RA) Right','League Rank')] <= 10) & (row[('In The Paint (Non-RA) Right','League Rank')] > 0):
        style[12] = 'background-color: gold'
    else:
        style[12] = 'background-color: transparent'
        
    if (row[('Mid-Range Center','League Rank')] <= 10) & (row[('Mid-Range Center','League Rank')] > 0):
        style[17] = 'background-color: gold'
    else:
        style[17] = 'background-color: transparent'
        
    if (row[('Mid-Range Left Center','League Rank')] <= 10) & (row[('Mid-Range Left Center','League Rank')] > 0):
        style[22] = 'background-color: gold'
    else:
        style[22] = 'background-color: transparent'
        
    if (row[('Mid-Range Left','League Rank')] <= 10) & (row[('Mid-Range Left','League Rank')] > 0):
        style[27] = 'background-color: gold'
    else:
        style[27] = 'background-color: transparent'
        
    if (row[('Mid-Range Right Center','League Rank')] <= 10) & (row[('Mid-Range Right Center','League Rank')] > 0):
        style[32] = 'background-color: gold'
    else:
        style[32] = 'background-color: transparent'
        
    if (row[('Mid-Range Right','League Rank')] <= 10) & (row[('Mid-Range Right','League Rank')] > 0):
        style[37] = 'background-color: gold'
    else:
        style[37] = 'background-color: transparent'
        
    if (row[('Restricted Area','League Rank')] <= 10) & (row[('Restricted Area','League Rank')] > 0):
        style[42] = 'background-color: gold'
    else:
        style[42] = 'background-color: transparent'
        
    if (row[('3PT Above the Break Center','League Rank')] <= 10) & (row[('3PT Above the Break Center','League Rank')] > 0):
        style[47] = 'background-color: gold'
    else:
        style[47] = 'background-color: transparent'
        
    if (row[('3PT Above the Break Left Center','League Rank')] <= 10) & (row[('3PT Above the Break Left Center','League Rank')] > 0):
        style[52] = 'background-color: gold'
    else:
        style[52] = 'background-color: transparent'
        
    if (row[('3PT Above the Break Right Center','League Rank')] <= 10) & (row[('3PT Above the Break Right Center','League Rank')] > 0):
        style[57] = 'background-color: gold'
    else:
        style[57] = 'background-color: transparent'

    if (row[('3PT Left Corner','League Rank')] <= 10) & (row[('3PT Left Corner','League Rank')] > 0):
        style[62] = 'background-color: gold'
    else:
        style[62] = 'background-color: transparent'
        
    if (row[('3PT Right Corner','League Rank')] <= 10) & (row[('3PT Right Corner','League Rank')] > 0):
        style[67] = 'background-color: gold'
    else:
        style[67] = 'background-color: transparent'
        
    if (row[('2PT','League Rank')] <= 10) & (row[('2PT','League Rank')] > 0):
        style[72] = 'background-color: gold'
    else:
        style[72] = 'background-color: transparent'
        
    if (row[('3PT','League Rank')] <= 10) & (row[('3PT','League Rank')] > 0):
        style[77] = 'background-color: gold'
    else:
        style[77] = 'background-color: transparent'
        
    if (row[('Total','League Rank')] <= 10) & (row[('Total','League Rank')] > 0):
        style[82] = 'background-color: gold'
    else:
        style[82] = 'background-color: transparent'
    
    
    return style

# Created the below parent class and the subsequent child classes below
# so I could pull the individual players into their current rosters

class NBA_Team_Stats_With_ShotCharts:      
    
    def __init__(self, team):
        self.team = teams.get_teams()[0]
  
    def Team_ShotChart_Table(self):    
        Roster = self.team
        roster_ids = Roster['id']
        org = commonteamroster.CommonTeamRoster(team_id=roster_ids, season='2019-20', headers=STATS_HEADERS)
        organ = org.get_data_frames()[0]
        organization = pd.DataFrame(organ,columns=['PLAYER', 'PLAYER_ID', 'TeamID'])
        organization.rename(columns = {'PLAYER':'PLAYER_NAME'}, inplace = True)
        
        organization.set_index('PLAYER_NAME', inplace=True)
        
        Team_ShotDetail = pd.merge(All_ShotChart_Table(), organization, on='PLAYER_NAME')
        
        Team_ShotDetail_1 = Team_ShotDetail.replace(np.nan, 0)

        Team_ShotDetail_2 = Team_ShotDetail_1.drop(['PLAYER_ID','TeamID'], axis=1)
        
        Team_ShotDetail_2.columns = Team_ShotDetail_2.columns.str.split('_',expand=True)
    
        return (Team_ShotDetail_2.style
        .apply(custom_style, axis=1)
        .background_gradient(cmap='coolwarm', subset=[('Total','Percentile'), ('2PT','Percentile'), ('In The Paint (Non-RA) Right','Percentile'),
                                                     ('In The Paint (Non-RA) Left','Percentile'), ('In The Paint (Non-RA) Center','Percentile'),
                                                     ('Mid-Range Center','Percentile'), ('Mid-Range Left','Percentile'), ('Mid-Range Left Center','Percentile'),
                                                     ('Mid-Range Right','Percentile'), ('Mid-Range Right Center','Percentile'), ('Restricted Area','Percentile'),
                                                     ('3PT Above the Break Center','Percentile'), ('3PT Above the Break Right Center','Percentile'), ('3PT Above the Break Left Center','Percentile'),
                                                     ('3PT Left Corner','Percentile'), ('3PT Right Corner','Percentile'), ('3PT','Percentile')])
         
        .applymap(lambda v: 'background-color: %s' % 'transparent' if v==0 else '', subset=[('Total','Percentile'), ('2PT','Percentile'), ('In The Paint (Non-RA) Right','Percentile'),
                                                     ('In The Paint (Non-RA) Left','Percentile'), ('In The Paint (Non-RA) Center','Percentile'),
                                                     ('Mid-Range Center','Percentile'), ('Mid-Range Left','Percentile'), ('Mid-Range Left Center','Percentile'),
                                                     ('Mid-Range Right','Percentile'), ('Mid-Range Right Center','Percentile'), ('Restricted Area','Percentile'),
                                                     ('3PT Above the Break Center','Percentile'), ('3PT Above the Break Right Center','Percentile'), ('3PT Above the Break Left Center','Percentile'),
                                                     ('3PT Left Corner','Percentile'), ('3PT Right Corner','Percentile'), ('3PT','Percentile')])
        
        .hide_columns([('Total','League Rank'), ('2PT','League Rank'), ('In The Paint (Non-RA) Right','League Rank'),('In The Paint (Non-RA) Left','League Rank'), ('In The Paint (Non-RA) Center','League Rank'),
                       ('Mid-Range Center','League Rank'), ('Mid-Range Left','League Rank'), ('Mid-Range Left Center','League Rank'),('Mid-Range Right','League Rank'), ('Mid-Range Right Center','League Rank'), ('Restricted Area','League Rank'),
                       ('3PT Above the Break Center','League Rank'), ('3PT Above the Break Right Center','League Rank'), ('3PT Above the Break Left Center','League Rank'),('3PT Left Corner','League Rank'), ('3PT Right Corner','League Rank'),
                       ('3PT','League Rank'),('In The Paint (Non-RA) 2PT Center(C) Made','nan'),('In The Paint (Non-RA) 2PT Center(C) Attempted','nan'),('In The Paint (Non-RA) 2PT Left Side(L) Made','nan'),
                       ('In The Paint (Non-RA) 2PT Left Side(L) Attempted','nan'),('In The Paint (Non-RA) 2PT Right Side(R) Made','nan'),('In The Paint (Non-RA) 2PT Right Side(R) Attempted','nan'),
                       ('Mid-Range 2PT Center(C) Made','nan'),('Mid-Range 2PT Center(C) Attempted','nan'),('Mid-Range 2PT Left Side Center(LC) Made','nan'),('Mid-Range 2PT Left Side Center(LC) Attempted','nan'),
                       ('Mid-Range 2PT Left Side(L) Made','nan'),('Mid-Range 2PT Left Side(L) Attempted','nan'),('Mid-Range 2PT Right Side Center(RC) Made','nan'),('Mid-Range 2PT Right Side Center(RC) Attempted','nan'),
                       ('Mid-Range 2PT Right Side(R) Made','nan'),('Mid-Range 2PT Right Side(R) Attempted','nan'),('Restricted Area 2PT Center(C) Made','nan'),('Restricted Area 2PT Center(C) Attempted','nan'),('Above the Break 3PT Center(C) Made','nan'),
                       ('Above the Break 3PT Center(C) Attempted','nan'),('Above the Break 3PT Left Side Center(LC) Made','nan'),('Above the Break 3PT Left Side Center(LC) Attempted','nan'),('Above the Break 3PT Right Side Center(RC) Made','nan'),
                       ('Above the Break 3PT Right Side Center(RC) Attempted','nan'),('Left Corner 3PT Left Side(L) Made','nan'),('Left Corner 3PT Left Side(L) Attempted','nan'),('Right Corner 3PT Right Side(R) Made','nan'),('Right Corner 3PT Right Side(R) Attempted','nan'),
                       ('2PT Field Goals Made','nan'),('2PT Field Goals Attempted','nan'),('3PT Field Goals Made','nan'),('3PT Field Goals Attempted','nan'),('Total Made','nan'),('Total Attempts','nan')])
                
         .format({('Total','%'): '{:.0%}'.format,
                 ('Total','Percentile'): '{:.0f}'.format,
                 ('In The Paint (Non-RA) Center','%'): '{:.0%}'.format,
                 ('In The Paint (Non-RA) Center','Percentile'): '{:.0f}'.format,
                 ('In The Paint (Non-RA) Right','%'): '{:.0%}'.format,
                 ('In The Paint (Non-RA) Right','Percentile'): '{:.0f}'.format,
                 ('In The Paint (Non-RA) Left','%'): '{:.0%}'.format,
                 ('In The Paint (Non-RA) Left','Percentile'): '{:.0f}'.format,
                 ('Mid-Range Center','%'): '{:.0%}'.format,
                 ('Mid-Range Center','Percentile'): '{:.0f}'.format,
                 ('Mid-Range Right','%'): '{:.0%}'.format,
                 ('Mid-Range Right','Percentile'): '{:.0f}'.format,
                 ('Mid-Range Right Center','%'): '{:.0%}'.format,
                 ('Mid-Range Right Center','Percentile'): '{:.0f}'.format,
                 ('Mid-Range Left','%'): '{:.0%}'.format,
                 ('Mid-Range Left','Percentile'): '{:.0f}'.format,
                 ('Mid-Range Left Center','%'): '{:.0%}'.format,
                 ('Mid-Range Left Center','Percentile'): '{:.0f}'.format,
                 ('Restricted Area','%'): '{:.0%}'.format,
                 ('Restricted Area','Percentile'): '{:.0f}'.format,
                 ('3PT Above the Break Center','%'): '{:.0%}'.format,
                 ('3PT Above the Break Center','Percentile'): '{:.0f}'.format,
                 ('3PT Above the Break Right Center','%'): '{:.0%}'.format,
                 ('3PT Above the Break Right Center','Percentile'): '{:.0f}'.format,
                 ('3PT Above the Break Left Center','%'): '{:.0%}'.format,
                 ('3PT Above the Break Left Center','Percentile'): '{:.0f}'.format,
                 ('3PT Above the Break Left Center','%'): '{:.0%}'.format,
                 ('3PT Left Corner','%'): '{:.0%}'.format,
                 ('3PT Left Corner','Percentile'): '{:.0f}'.format,
                 ('3PT Right Corner','%'): '{:.0%}'.format,
                 ('3PT Right Corner','Percentile'): '{:.0f}'.format,
                 ('2PT','%'): '{:.0%}'.format,
                 ('2PT','Percentile'): '{:.0f}'.format,
                 ('3PT','%'): '{:.0%}'.format,
                 ('3PT','Percentile'): '{:.0f}'.format}))

        
class Atlanta_Hawks(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        super().__init__(team)
        
class Boston_Celtics(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[1]
        
class Cleveland_Cavaliers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[2]
        
class New_Orleans_Pelicans(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[3]
        
class Chicago_Bulls(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[4]
        
class Dallas_Mavericks(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[5]
        
class Denver_Nuggets(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[6]
        
class GS_Warriors(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[7]
        
class Houston_Rockets(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[8]
        
class LA_Clippers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[9]
        
class LA_Lakers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[10]
        
class Miami_Heat(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[11]
        
class Milwaukee_Bucks(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[12]
        
class Minnesota_Timberwolves(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[13]
        
class Brooklyn_Nets(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[14]
        
class NY_Knicks(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[15]
        
class Orlando_Magic(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[16]
        
class Indiana_Pacers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[17]
        
class Philadelphia_76ers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[18]
        
class Phoenix_Suns(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[19]
        
class Portland_Trailblazers(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[20]
        
class Sacramento_Kings(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[21]
        
class San_Antonio_Spurs(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[22]
        
class Oklahoma_City_Thunder(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[23]
        
class Toronto_Raptors(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[24]
        
class Utah_Jazz(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[25]
        
class Memphis_Grizzles(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[26]
        
class Washington_Wizards(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[27]
        
class Detroit_Pistons(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[28]
        
class Charlotte_Hornets(NBA_Team_Stats_With_ShotCharts):
    def __init__(self, team):
        self.team = teams.get_teams()[29]

Atlanta_Hawks = Atlanta_Hawks("Hawks")
Boston_Celtics = Boston_Celtics("Celtics")
Cleveland_Cavaliers = Cleveland_Cavaliers("Cavs")
New_Orleans_Pelicans = New_Orleans_Pelicans("Pelicans")
Charlotte_Hornets = Charlotte_Hornets("Hornets")
Chicago_Bulls = Chicago_Bulls("Bulls")
Dallas_Mavericks = Dallas_Mavericks("Mavericks")
Denver_Nuggets = Denver_Nuggets("Nuggets")
GS_Warriors = GS_Warriors("Warriors")
Houston_Rockets = Houston_Rockets("Rockets")
LA_Clippers = LA_Clippers("Clippers")
LA_Lakers = LA_Lakers("Lakers")
Miami_Heat = Miami_Heat("Heat")
Milwaukee_Bucks = Milwaukee_Bucks("Bucks")
Minnesota_Timberwolves = Minnesota_Timberwolves("Timberwolves")
Brooklyn_Nets = Brooklyn_Nets("Nets")
NY_Knicks = NY_Knicks("Knicks")
Orlando_Magic = Orlando_Magic("Magic")
Indiana_Pacers = Indiana_Pacers("Pacers")
Philadelphia_76ers = Philadelphia_76ers("76ers")
Phoenix_Suns = Phoenix_Suns("Suns")
Portland_Trailblazers = Portland_Trailblazers("Blazers")
Sacramento_Kings = Sacramento_Kings("Kings")
San_Antonio_Spurs = San_Antonio_Spurs("Spurs")
Oklahoma_City_Thunder = Oklahoma_City_Thunder("Thunder")
Toronto_Raptors = Toronto_Raptors("Raptors")
Utah_Jazz = Utah_Jazz("Jazz")
Memphis_Grizzles = Memphis_Grizzles("Grizzles")
Washington_Wizards = Washington_Wizards("Wizards")
Detroit_Pistons = Detroit_Pistons("Pistons")

# Those players with blank Percentile Rank cells did not meet the minimum shot attempt requirement for that location

In [25]:
LA_Clippers.Team_ShotChart_Table()

In [26]:
data = LA_Clippers.Team_ShotChart_Table()

import random
import imgkit
import pandas
import os

html = data.render()

def DataFrame_to_image(data, css, outputfile="LA Clippers.png", format="png"):
    '''
    For rendering a Pandas DataFrame as an image.
    data: a pandas DataFrame
    css: a string containing rules for styling the output table. This must 
         contain both the opening an closing <style> tags.
    *outputimage: filename for saving of generated image
    *format: output format, as supported by IMGKit. Default is "png"
    '''
    fn = str(random.random()*100000000).split(".")[0] + ".html"
    
    try:
        os.remove(fn)
    except:
        None
    text_file = open(fn, "a")
    
    # write the CSS
    text_file.write(css)
    # write the HTML-ized Pandas DataFrame
    text_file.write(data.render())
    text_file.close()
    
    # See IMGKit options for full configuration,
    # e.g. cropping of final image
    imgkitoptions = {"format": format}
    
    imgkit.from_file(fn, outputfile, options=imgkitoptions)
    os.remove(fn)
    
    
css = """
<style type=\"text/css\">
table {
color: #333;
font-family: Helvetica, Arial, sans-serif;
width: 640px;
border-collapse:
collapse; 
border-spacing: 0;
}

td, th {
border: medium solid black; /* No more visible border */
height: 30px;
}

th {
background: #DFDFDF; /* Darken header a bit */
font-weight: bold;
}

td {
background: #FAFAFA;
text-align: center;
}

table tr:nth-child(odd) td{
background-color: white;
}
</style>
"""

DataFrame_to_image(data, css)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
